In [14]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import LabelEncoder
import subprocess

### Load Data

In [2]:
# df_train = pd.read_pickle("input/parsed_train.pkl")
# df_test = pd.read_pickle("input/parsed_test.pkl")

df_train = pd.read_csv("input/parsed_train.csv", dtype={'fullVisitorId': 'str'})
df_test = pd.read_csv("input/parsed_test.csv", dtype={'fullVisitorId': 'str'})

/Users/hyundai/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Feature Engineering for df_train

In [4]:
# df_train = df_train[:10000]
# df_test = df_test[:10000]

### Split X and y, Remove not_used_cols

In [5]:
target_col = 'totals_transactionRevenue'
df_train_y = df_train[target_col].astype(float)
df_train_y.fillna(0, inplace=True)
df_train_y = np.log1p(df_train_y)

not_used_cols = [
    "visitNumber", "date", "fullVisitorId", "sessionId", "visitId",
    "visitStartTime", 'trafficSource_referralPath',
    'trafficSource_campaignCode', target_col,
]
df_train_X = df_train.drop(not_used_cols, axis=1)

df_test_idx = df_test[['fullVisitorId']]
del_cols = list(set(df_test.columns) & set(not_used_cols))
df_test_X = df_test.drop(del_cols, axis=1)

In [6]:
print(df_train_X.columns)
print(df_test_X.columns)

Index(['channelGrouping', 'socialEngagementType', 'device_browser',
       'device_browserSize', 'device_browserVersion', 'device_deviceCategory',
       'device_flashVersion', 'device_isMobile', 'device_language',
       'device_mobileDeviceBranding', 'device_mobileDeviceInfo',
       'device_mobileDeviceMarketingName', 'device_mobileDeviceModel',
       'device_mobileInputSelector', 'device_operatingSystem',
       'device_operatingSystemVersion', 'device_screenColors',
       'device_screenResolution', 'geoNetwork_city', 'geoNetwork_cityId',
       'geoNetwork_continent', 'geoNetwork_country', 'geoNetwork_latitude',
       'geoNetwork_longitude', 'geoNetwork_metro', 'geoNetwork_networkDomain',
       'geoNetwork_networkLocation', 'geoNetwork_region',
       'geoNetwork_subContinent', 'totals_bounces', 'totals_hits',
       'totals_newVisits', 'totals_pageviews', 'totals_visits',
       'trafficSource_adContent',
       'trafficSource_adwordsClickInfo.adNetworkType',
       'trafficS

### Label Encoding

In [7]:
categorical_feature = list(df_train_X.select_dtypes(exclude=np.number).columns)
print(categorical_feature)

df_train_X[categorical_feature].fillna('NA_NULL', inplace=True)
df_train_X.fillna(0, inplace=True)

df_test_X[categorical_feature].fillna('NA_NULL', inplace=True)
df_test_X.fillna(0, inplace=True)

for c in categorical_feature:
#     print(c)
    lbl = LabelEncoder()
    lbl.fit(
        list(df_train_X[c].values.astype('str')) +
        list(df_test_X[c].values.astype('str')))
    df_train_X[c] = lbl.transform(list(df_train_X[c].values.astype('str')))
    df_test_X[c] = lbl.transform(list(df_test_X[c].values.astype('str')))

['channelGrouping', 'socialEngagementType', 'device_browser', 'device_browserSize', 'device_browserVersion', 'device_deviceCategory', 'device_flashVersion', 'device_isMobile', 'device_language', 'device_mobileDeviceBranding', 'device_mobileDeviceInfo', 'device_mobileDeviceMarketingName', 'device_mobileDeviceModel', 'device_mobileInputSelector', 'device_operatingSystem', 'device_operatingSystemVersion', 'device_screenColors', 'device_screenResolution', 'geoNetwork_city', 'geoNetwork_cityId', 'geoNetwork_continent', 'geoNetwork_country', 'geoNetwork_latitude', 'geoNetwork_longitude', 'geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_networkLocation', 'geoNetwork_region', 'geoNetwork_subContinent', 'trafficSource_adContent', 'trafficSource_adwordsClickInfo.adNetworkType', 'trafficSource_adwordsClickInfo.criteriaParameters', 'trafficSource_adwordsClickInfo.gclId', 'trafficSource_adwordsClickInfo.isVideoAd', 'trafficSource_adwordsClickInfo.slot', 'trafficSource_campaign', 'traffic

/Users/hyundai/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [20]:
df_val_X = df_train_X
df_val_y = df_train_y

feature_name = list(df_train_X.columns)

train = lgb.Dataset(
    df_train_X.values,
    label=df_train_y.values,
    feature_name=feature_name,
    categorical_feature=categorical_feature)

valid = lgb.Dataset(
    df_val_X.values,
    label=df_val_y.values,
    feature_name=feature_name,
    categorical_feature=categorical_feature)

params = {
    "objective": "regression",
    "metric": "rmse",
    "max_depth": 8,
    "min_child_samples": 20,
    "reg_alpha": 1,
    "reg_lambda": 1,
    "num_leaves": 257,
    "learning_rate": 0.01,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "subsample_freq ": 5,
}

evals_result = {}
train_params = {
    'params': params,
    'train_set': train,
    'valid_sets': [train, valid],
    'valid_names': ['train', 'valid'],
    'evals_result': evals_result,
    'early_stopping_rounds': 100,  #50
    'num_boost_round': 20000,  #500
    'verbose_eval': 10,
    'feval': None,
    #     'categorical_feature': 'auto',
}

model = lgb.train(**train_params)

n_estimators = model.best_iteration

/Users/hyundai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1158: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/hyundai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:725: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[10]	train's rmse: 1.94688	valid's rmse: 1.94688
[20]	train's rmse: 1.89812	valid's rmse: 1.89812
[30]	train's rmse: 1.85609	valid's rmse: 1.85609
[40]	train's rmse: 1.82592	valid's rmse: 1.82592
[50]	train's rmse: 1.79945	valid's rmse: 1.79945
[60]	train's rmse: 1.77201	valid's rmse: 1.77201
[70]	train's rmse: 1.74841	valid's rmse: 1.74841
[80]	train's rmse: 1.72816	valid's rmse: 1.72816
[90]	train's rmse: 1.7114	valid's rmse: 1.7114
[100]	train's rmse: 1.69601	valid's rmse: 1.69601
[110]	train's rmse: 1.6824	valid's rmse: 1.6824
[120]	train's rmse: 1.67099	valid's rmse: 1.67099
[130]	train's rmse: 1.66128	valid's rmse: 1.66128
[140]	train's rmse: 1.65264	valid's rmse: 1.65264
[150]	train's rmse: 1.64498	valid's rmse: 1.64498
[160]	train's rmse: 1.6384	valid's rmse: 1.6384
[170]	train's rmse: 1.63172	valid's rmse: 1.63172
[180]	train's rmse: 1.62598	valid's rmse: 1.62598
[190]	train's rmse: 1.62084	valid's rmse: 1.62084
[2

[1640]	train's rmse: 1.51171	valid's rmse: 1.51171
[1650]	train's rmse: 1.51149	valid's rmse: 1.51149
[1660]	train's rmse: 1.51136	valid's rmse: 1.51136
[1670]	train's rmse: 1.51117	valid's rmse: 1.51117
[1680]	train's rmse: 1.5108	valid's rmse: 1.5108
[1690]	train's rmse: 1.51056	valid's rmse: 1.51056
[1700]	train's rmse: 1.5104	valid's rmse: 1.5104
[1710]	train's rmse: 1.5102	valid's rmse: 1.5102
[1720]	train's rmse: 1.51002	valid's rmse: 1.51002
[1730]	train's rmse: 1.50985	valid's rmse: 1.50985
[1740]	train's rmse: 1.50955	valid's rmse: 1.50955
[1750]	train's rmse: 1.5093	valid's rmse: 1.5093
[1760]	train's rmse: 1.50909	valid's rmse: 1.50909
[1770]	train's rmse: 1.50885	valid's rmse: 1.50885
[1780]	train's rmse: 1.50875	valid's rmse: 1.50875
[1790]	train's rmse: 1.5086	valid's rmse: 1.5086
[1800]	train's rmse: 1.50837	valid's rmse: 1.50837
[1810]	train's rmse: 1.50805	valid's rmse: 1.50805
[1820]	train's rmse: 1.50773	valid's rmse: 1.50773
[1830]	train's rmse: 1.50747	valid's rmse

[3260]	train's rmse: 1.48268	valid's rmse: 1.48268
[3270]	train's rmse: 1.48246	valid's rmse: 1.48246
[3280]	train's rmse: 1.48232	valid's rmse: 1.48232
[3290]	train's rmse: 1.48218	valid's rmse: 1.48218
[3300]	train's rmse: 1.48203	valid's rmse: 1.48203
[3310]	train's rmse: 1.48184	valid's rmse: 1.48184
[3320]	train's rmse: 1.48174	valid's rmse: 1.48174
[3330]	train's rmse: 1.48154	valid's rmse: 1.48154
[3340]	train's rmse: 1.48137	valid's rmse: 1.48137
[3350]	train's rmse: 1.48124	valid's rmse: 1.48124
[3360]	train's rmse: 1.48112	valid's rmse: 1.48112
[3370]	train's rmse: 1.48096	valid's rmse: 1.48096
[3380]	train's rmse: 1.48082	valid's rmse: 1.48082
[3390]	train's rmse: 1.48064	valid's rmse: 1.48064
[3400]	train's rmse: 1.48047	valid's rmse: 1.48047
[3410]	train's rmse: 1.48038	valid's rmse: 1.48038
[3420]	train's rmse: 1.48019	valid's rmse: 1.48019
[3430]	train's rmse: 1.48008	valid's rmse: 1.48008
[3440]	train's rmse: 1.47991	valid's rmse: 1.47991
[3450]	train's rmse: 1.4798	val

[4880]	train's rmse: 1.46087	valid's rmse: 1.46087
[4890]	train's rmse: 1.46084	valid's rmse: 1.46084
[4900]	train's rmse: 1.46078	valid's rmse: 1.46078
[4910]	train's rmse: 1.46064	valid's rmse: 1.46064
[4920]	train's rmse: 1.46058	valid's rmse: 1.46058
[4930]	train's rmse: 1.46048	valid's rmse: 1.46048
[4940]	train's rmse: 1.46039	valid's rmse: 1.46039
[4950]	train's rmse: 1.46034	valid's rmse: 1.46034
[4960]	train's rmse: 1.46027	valid's rmse: 1.46027
[4970]	train's rmse: 1.46017	valid's rmse: 1.46017
[4980]	train's rmse: 1.46013	valid's rmse: 1.46013
[4990]	train's rmse: 1.46001	valid's rmse: 1.46001
[5000]	train's rmse: 1.45982	valid's rmse: 1.45982
[5010]	train's rmse: 1.4597	valid's rmse: 1.4597
[5020]	train's rmse: 1.45965	valid's rmse: 1.45965
[5030]	train's rmse: 1.4595	valid's rmse: 1.4595
[5040]	train's rmse: 1.45939	valid's rmse: 1.45939
[5050]	train's rmse: 1.45919	valid's rmse: 1.45919
[5060]	train's rmse: 1.459	valid's rmse: 1.459
[5070]	train's rmse: 1.45881	valid's rm

...

[12920]	train's rmse: 1.38264	valid's rmse: 1.38264
[12930]	train's rmse: 1.38252	valid's rmse: 1.38252
[12940]	train's rmse: 1.38244	valid's rmse: 1.38244
[12950]	train's rmse: 1.38232	valid's rmse: 1.38232
[12960]	train's rmse: 1.38225	valid's rmse: 1.38225
[12970]	train's rmse: 1.38206	valid's rmse: 1.38206
[12980]	train's rmse: 1.38194	valid's rmse: 1.38194
[12990]	train's rmse: 1.38179	valid's rmse: 1.38179
[13000]	train's rmse: 1.38162	valid's rmse: 1.38162
[13010]	train's rmse: 1.38151	valid's rmse: 1.38151
[13020]	train's rmse: 1.38137	valid's rmse: 1.38137
[13030]	train's rmse: 1.38124	valid's rmse: 1.38124
[13040]	train's rmse: 1.38109	valid's rmse: 1.38109
[13050]	train's rmse: 1.38097	valid's rmse: 1.38097
[13060]	train's rmse: 1.38083	valid's rmse: 1.38083
[13070]	train's rmse: 1.38061	valid's rmse: 1.38061
[13080]	train's rmse: 1.38044	valid's rmse: 1.38044
[13090]	train's rmse: 1.38031	valid's rmse: 1.38031
[13100]	train's rmse: 1.38017	valid's rmse: 1.38017
[13110]	trai

[14510]	train's rmse: 1.36806	valid's rmse: 1.36806
[14520]	train's rmse: 1.36794	valid's rmse: 1.36794
[14530]	train's rmse: 1.36779	valid's rmse: 1.36779
[14540]	train's rmse: 1.36773	valid's rmse: 1.36773
[14550]	train's rmse: 1.36766	valid's rmse: 1.36766
[14560]	train's rmse: 1.36754	valid's rmse: 1.36754
[14570]	train's rmse: 1.36741	valid's rmse: 1.36741
[14580]	train's rmse: 1.36721	valid's rmse: 1.36721
[14590]	train's rmse: 1.36706	valid's rmse: 1.36706
[14600]	train's rmse: 1.36694	valid's rmse: 1.36694
[14610]	train's rmse: 1.36679	valid's rmse: 1.36679
[14620]	train's rmse: 1.36669	valid's rmse: 1.36669
[14630]	train's rmse: 1.36649	valid's rmse: 1.36649
[14640]	train's rmse: 1.36635	valid's rmse: 1.36635
[14650]	train's rmse: 1.36616	valid's rmse: 1.36616
[14660]	train's rmse: 1.366	valid's rmse: 1.366
[14670]	train's rmse: 1.36585	valid's rmse: 1.36585
[14680]	train's rmse: 1.36574	valid's rmse: 1.36574
[14690]	train's rmse: 1.3656	valid's rmse: 1.3656
[14700]	train's rm

[16100]	train's rmse: 1.3507	valid's rmse: 1.3507
[16110]	train's rmse: 1.35061	valid's rmse: 1.35061
[16120]	train's rmse: 1.35057	valid's rmse: 1.35057
[16130]	train's rmse: 1.35054	valid's rmse: 1.35054
[16140]	train's rmse: 1.3505	valid's rmse: 1.3505
[16150]	train's rmse: 1.35048	valid's rmse: 1.35048
[16160]	train's rmse: 1.35045	valid's rmse: 1.35045
[16170]	train's rmse: 1.35043	valid's rmse: 1.35043
[16180]	train's rmse: 1.35041	valid's rmse: 1.35041
[16190]	train's rmse: 1.35038	valid's rmse: 1.35038
[16200]	train's rmse: 1.35035	valid's rmse: 1.35035
[16210]	train's rmse: 1.35033	valid's rmse: 1.35033
[16220]	train's rmse: 1.35031	valid's rmse: 1.35031
[16230]	train's rmse: 1.35027	valid's rmse: 1.35027
[16240]	train's rmse: 1.35026	valid's rmse: 1.35026
[16250]	train's rmse: 1.35017	valid's rmse: 1.35017
[16260]	train's rmse: 1.35009	valid's rmse: 1.35009
[16270]	train's rmse: 1.35004	valid's rmse: 1.35004
[16280]	train's rmse: 1.34997	valid's rmse: 1.34997
[16290]	train's 

[17690]	train's rmse: 1.34045	valid's rmse: 1.34045
[17700]	train's rmse: 1.34042	valid's rmse: 1.34042
[17710]	train's rmse: 1.34037	valid's rmse: 1.34037
[17720]	train's rmse: 1.34023	valid's rmse: 1.34023
[17730]	train's rmse: 1.34013	valid's rmse: 1.34013
[17740]	train's rmse: 1.34001	valid's rmse: 1.34001
[17750]	train's rmse: 1.33988	valid's rmse: 1.33988
[17760]	train's rmse: 1.33978	valid's rmse: 1.33978
[17770]	train's rmse: 1.3397	valid's rmse: 1.3397
[17780]	train's rmse: 1.33962	valid's rmse: 1.33962
[17790]	train's rmse: 1.33953	valid's rmse: 1.33953
[17800]	train's rmse: 1.33939	valid's rmse: 1.33939
[17810]	train's rmse: 1.33935	valid's rmse: 1.33935
[17820]	train's rmse: 1.33922	valid's rmse: 1.33922
[17830]	train's rmse: 1.3391	valid's rmse: 1.3391
[17840]	train's rmse: 1.33899	valid's rmse: 1.33899
[17850]	train's rmse: 1.33891	valid's rmse: 1.33891
[17860]	train's rmse: 1.3388	valid's rmse: 1.3388
[17870]	train's rmse: 1.33872	valid's rmse: 1.33872
[17880]	train's rm

[19280]	train's rmse: 1.32893	valid's rmse: 1.32893
[19290]	train's rmse: 1.32891	valid's rmse: 1.32891
[19300]	train's rmse: 1.3289	valid's rmse: 1.3289
[19310]	train's rmse: 1.32886	valid's rmse: 1.32886
[19320]	train's rmse: 1.32882	valid's rmse: 1.32882
[19330]	train's rmse: 1.32878	valid's rmse: 1.32878
[19340]	train's rmse: 1.3287	valid's rmse: 1.3287
[19350]	train's rmse: 1.32868	valid's rmse: 1.32868
[19360]	train's rmse: 1.32866	valid's rmse: 1.32866
[19370]	train's rmse: 1.32862	valid's rmse: 1.32862
[19380]	train's rmse: 1.32853	valid's rmse: 1.32853
[19390]	train's rmse: 1.32843	valid's rmse: 1.32843
[19400]	train's rmse: 1.32834	valid's rmse: 1.32834
[19410]	train's rmse: 1.32823	valid's rmse: 1.32823
[19420]	train's rmse: 1.32815	valid's rmse: 1.32815
[19430]	train's rmse: 1.32805	valid's rmse: 1.32805
[19440]	train's rmse: 1.32795	valid's rmse: 1.32795
[19450]	train's rmse: 1.3279	valid's rmse: 1.3279
[19460]	train's rmse: 1.32779	valid's rmse: 1.32779
[19470]	train's rm

### Feature Engineering for df_test

### Predict and submit

In [21]:
pred = np.zeros(df_test_X.shape[0])
y_pred = model.predict(df_test_X, num_iteration=model.best_iteration)
pred += y_pred

df_test_idx['PredictedLogRevenue'] = pred
df_test_idx["PredictedLogRevenue"] = df_test_idx["PredictedLogRevenue"].apply(lambda x : 0.0 if x < 0 else x)
df_test_idx["PredictedLogRevenue"] = df_test_idx["PredictedLogRevenue"].fillna(0.0)
df_submit = df_test_idx[['fullVisitorId', 'PredictedLogRevenue']].groupby('fullVisitorId').sum().reset_index()
df_submit.head(10)

/Users/hyundai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/hyundai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/hyundai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pan

,fullVisitorId,PredictedLogRevenue
0,0000000259678714014,0.001550
1,0000049363351866189,0.004380
2,0000053049821714864,0.011387
3,0000059488412965267,0.000000
4,0000085840370633780,0.019182
5,0000091131414287111,0.000918
6,0000117255350596610,5.688907
7,0000118334805178127,0.000000
8,0000130646294093000,0.000240
9,0000150005271820273,0.001672


In [22]:
file_name = 'aiden_{}.csv.tar.gz'.format(datetime.datetime.now().strftime('%Y-%m-%dT%H%M%S'))
df_submit.to_csv(file_name, index=False, compression='gzip')
file_name

In [23]:
del train_params['evals_result']
del train_params['valid_sets']
msg = str(train_params)

In [28]:
cmd = """kaggle competitions submit -c ga-customer-revenue-prediction -f {file_path} -m "{msg}"
""".format(**{'file_path': file_name, 'msg': msg})
print(cmd)
subprocess.call(cmd, shell=True)

kaggle competitions submit -c ga-customer-revenue-prediction -f aiden_2018-09-23T005219.csv.tar.gz -m "msg"



0

In [13]:
!kaggle competitions submit ga-customer-revenue-prediction -f aiden_2018-09-23T00:09:37.csv.tar.gz -m "My first summit"

Successfully submitted to Google Analytics Customer Revenue Prediction